# Original SynthSeg
env: python3.8 tensorflow

In [4]:
# import and constant
import os
import numpy as np
import nibabel as nib
from utils.nrrd2nii import nrrd2nii

source_path = "data/uwa_aaa"
combined_path = "data/uwa_aaa_combined"
# generation
image_path = "data/Dataset001_aaa/imagesTr"
label_path = "data/Dataset001_aaa/labelsTr"


# Generation Preprocess

In [ ]:

os.makedirs(combined_path, exist_ok=True)

for patient_folder in sorted(os.listdir(source_path))[:5]:
    patient_path = os.path.join(source_path, patient_folder)
    aaa_label = None
    blood_label = None
    for root, dirs, files in os.walk(patient_path):
        for file in files:
            if file == "CT_AAA_label.nrrd":
                aaa_label = nrrd2nii(os.path.join(root, file)).get_fdata()
                assert (
                    len(np.unique(aaa_label)) == 2
                ), f"Patient {patient_folder} label error"
                highest_label = np.max(aaa_label)
                aaa_label[aaa_label == highest_label] = 1

            if file == "CT_blood_label.nrrd":
                blood_label = nrrd2nii(os.path.join(root, file)).get_fdata()
                assert (
                    len(np.unique(blood_label)) == 2
                ), f"Patient {patient_folder} label error"
                highest_label = np.max(blood_label)
                blood_label[blood_label == highest_label] = 1

    assert (aaa_label is not None) and (
        blood_label is not None
    ), f"Patient {patient_folder} label uncompleted"

    combined_label = np.zeros_like(aaa_label)
    combined_label[aaa_label == 1] = 1
    combined_label[blood_label == 1] = 2
    
    # if shape contains singular, make it even
    padding = []
    for dim in combined_label.shape:
        if dim % 2 == 1:
            padding.append(1)
        else:
            padding.append(0)
    combined_label = np.pad(
        combined_label,
        ((0, padding[0]), (0, padding[1]), (0, padding[2])),
        mode="constant",
    )
    
    label_nii = nib.Nifti1Image(combined_label, np.eye(4))
    nib.save(label_nii, os.path.join(combined_path, f"{patient_folder}_combined.nii.gz"))

(110, 104, 126)
(88, 84, 100)
(120, 146, 232)
(236, 168, 246)
(172, 124, 280)


# Generation

In [5]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'synth_seg')))
from SynthSeg.brain_generator import BrainGenerator
# mkdir
os.makedirs(image_path, exist_ok=True)
os.makedirs(label_path, exist_ok=True)
label_files = os.listdir(combined_path)
count = 0
for label_file in label_files:
    print("here", os.path.join(combined_path, label_file))
    brain_generator = BrainGenerator(os.path.join(combined_path, label_file))
    for i in range(10):
        image, label = brain_generator.generate_brain()
        nib.save(
            nib.Nifti1Image(image, np.eye(4)),
            os.path.join(image_path, f"aaa_{count:03d}_0000.nii.gz"),
        )
        nib.save(
            nib.Nifti1Image(label, np.eye(4)),
            os.path.join(label_path, f"aaa_{count:03d}.nii.gz"),
        )
        count += 1

here data/uwa_aaa_combined/Patient1_combined.nii.gz
Compiling list of unique labels
processing 1/1


2025-03-31 14:15:41.022850: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2025-03-31 14:15:41.193891: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-31 14:15:41.193976: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3070 computeCapability: 8.6
coreClock: 1.755GHz coreCount: 46 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 417.29GiB/s
2025-03-31 14:15:41.194189: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2025-03-31 14:15:41.194284: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic 

# Inspection

In [ ]:

from utils.check_image import check_image
import nibabel as nib
from utils.nrrd2nii import nrrd2nii


label = nib.load("data/uwa_aaa_combined/Patient1_combined.nii.gz").get_fdata()
image = nrrd2nii("data/uwa_aaa/Patient1/Segmentation/CT_cropped.nrrd").get_fdata()

check_image(
    [
        {"image": label, "title": "label", "is_label": True},
        {"image": image, "title": "image", "is_label": False},
    ]
)

interactive(children=(IntSlider(value=0, description='slice_index_0', max=125), IntSlider(value=0, description…

In [7]:

from utils.check_image import check_image
import nibabel as nib
from utils.nrrd2nii import nrrd2nii


label = nib.load("data/Dataset001_aaa/labelsTr/aaa_000.nii.gz").get_fdata()
image = nib.load("data/Dataset001_aaa/imagesTr/aaa_000_0000.nii.gz").get_fdata()

check_image(
    [
        {"image": label, "title": "label", "is_label": True},
        {"image": image, "title": "image", "is_label": False},
    ]
)

interactive(children=(IntSlider(value=0, description='slice_index_0', max=125), IntSlider(value=0, description…